In [1]:
import numpy as np
import pandas as pd
# import re
# import usaddress

from address_normalize.paths import DATA_DIR

# Get matches
match_df = pd.read_csv(DATA_DIR / 'processed' / 'match_df.csv')


# Get raw data
raw_dir = DATA_DIR / 'raw'

files = list(raw_dir.glob('Master Project Address List*.xlsx'))
files.sort()

file_path = files[-1]

df_e911 = pd.read_excel(file_path, sheet_name='E911 Data', dtype={'GRAND LIST ZIPCODE': str})
main_df = pd.read_excel(file_path, sheet_name='Phase_I', dtype={'Zip Code': str})


Need to look up if match on Phase 1 matches E911, if so then move the following from E911 to Phase 1
- Owner 1
- Owner 2
- Grand List Address
- CAT (= category)
- GRAND LIST = ADDRESS column from N $\rightarrow$ Y

In [2]:
print(match_df.shape)
match_df.head()

(1975, 3)


,phase_1_address,score,e911_address
0,114 S BINGHAM ST,1.0,114 S BINGHAM ST
1,1896 OLD JERUSALEM RD,1.0,1896 OLD JERUSALEM RD
2,14 CHURCH ST,1.0,14 CHURCH ST
3,24 PARK LN,1.0,24 PARK Ln
4,156 MIDDLE RD,1.0,156 MIDDLE RD


In [3]:
# Remove the matches with score of 1 because they are pointless.
cleaned_matches_df = match_df[match_df['score'] < 1]
print(cleaned_matches_df.shape)
cleaned_matches_df.head()

(840, 3)


,phase_1_address,score,e911_address
482,10 HIBBARD HILL RD,0.666667,10 HIBBARD HILL
1105,123 LAMBERT LN,0.333333,123 LAMBEROUTE Ln
1106,16 ROBERTS AVE,0.333333,16 ROBEROUTES AVE
1107,28 ROBERTS AVE,0.333333,28 ROBEROUTES AVE
1108,37 ROBERTS AVE,0.333333,37 ROBEROUTES AVE


In [4]:
main_df.head()

,OBJECTID,REF,ESITEID,Build Phase,ADDRESS,E911 ADDRESS,OWNER 1,OWNER 2,GRAND LIST ADDRESS,GRAND LIST = ADDRESS,...,COL_R,COL_S,COL_T,COL_U,COL_V,COL_W,COL_X,COL_Y,GNS Area,Construction Status
0,1547,1547,78274,Phase 1,114 S BINGHAM ST,114 S BINGHAM ST,114 S BINGHAM LLC,(QUINTTUS),15 ASHLAND AVE,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Included
1,1316,1316,139583,Phase 1,1896 OLD JERUSALEM RD,1896 OLD JERUSALEM RD,80 GOATS INC,,2001 OLD JERUSALEM RD,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,Included
2,189,189,184321,Phase 1,14 CHURCH ST,14 CHURCH ST,ABBOTT JACOB,ABBOTT KAREN LYNN,14 CHURCH ST,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Included
3,1382,1382,78171,Phase 1,24 PARK LN,24 PARK Ln,ABEL WILLA & SIMON,,24 PARK LN,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GoNetSpeed,Excluded
4,328,328,216370,Phase 1,156 MIDDLE RD,156 MIDDLE RD,ACCIAVIATTI BRUCE,SMITH COLLEEN,156 MIDDLE RD,Y,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,Included


In [5]:
# Create a dictionary of the matches from phase 1 to e911
address_dict = dict(zip(cleaned_matches_df.phase_1_address, cleaned_matches_df.e911_address))

In [6]:
# # quick check on updates
# main_df['updated_e911_address'] = main_df['ADDRESS'].apply(lambda x: address_dict[x] if x in address_dict.keys() else None)

# print(main_df[(main_df['E911 ADDRESS'] != main_df['updated_e911_address']) & (~main_df['updated_e911_address'].isna())].shape)
# main_df[(main_df['E911 ADDRESS'] != main_df['updated_e911_address']) & (~main_df['updated_e911_address'].isna())].head(n=10)


In [7]:
main_df.columns

Index(['OBJECTID', 'REF', 'ESITEID', 'Build Phase', 'ADDRESS', 'E911 ADDRESS',
       'OWNER 1', 'OWNER 2', 'GRAND LIST ADDRESS ', 'GRAND LIST = ADDRESS',
       'CAT', 'Town', 'Zip Code', 'PON_Boundary', 'Status', 'Service_Speed',
       'Eligible', 'LAT', 'LONG', 'Source of Funding',
       'Act 71 Funded Overbuild', 'Funding for Drops if Overbuilt',
       'Overbuild Rationale', 'COL_Q', 'COL_R', 'COL_S', 'COL_T', 'COL_U',
       'COL_V', 'COL_W', 'COL_X', 'COL_Y', 'GNS Area', 'Construction Status'],
      dtype='object')

In [8]:
df_e911.columns

Index(['OBJECTID_1', 'E911 ADDRESS', 'OBJECTID', 'OWNER 1', 'OWNER 2',
       'GRAND LIST ADDRESS ', 'GRAND LIST CITY', 'GRAND LIST STATE',
       'GRAND LIST ZIPCODE', 'SPAN', 'GLIST_SPAN', 'MAPID', 'PARCID',
       'PROPTYPE', 'YEAR', 'GLYEAR', 'TOWN', 'TNAME', 'SOURCENAME',
       'SOURCETYPE', 'SOURCEDATE', 'EDITMETHOD', 'EDITOR', 'EDITDATE',
       'MATCHSTAT', 'EDITNOTE', 'ADDRGL2', 'DESCPROP', 'LOCAPROP', 'CAT',
       'RESCODE', 'ACRESGL', 'REAL_FLV', 'HSTED_FLV', 'NRES_FLV', 'LAND_LV',
       'IMPRV_LV', 'EQUIPVAL', 'EQUIPCODE', 'INVENVAL', 'HSDECL', 'HSITEVAL',
       'VETEXAMT', 'EXPDESC', 'ENDDATE', 'STATUTE', 'EXAMT_HS', 'EXAMT_NR',
       'UVREDUC_HS', 'UVREDUC_NR', 'GLVAL_HS', 'GLVAL_NR', 'CRHOUSPCT',
       'MUNGL1PCT', 'AOEGL_HS', 'AOEGL_NR', 'SHAPESTAre', 'SHAPESTLen',
       'Shape_Length', 'Shape_Area'],
      dtype='object')

In [9]:
def update_row(row: pd.Series):
    recorded_address = row['ADDRESS']
    recorded_zip = row['Zip Code']
    # print(recorded_address)
    if (recorded_address in address_dict.keys()) and (pd.isna(row['GRAND LIST = ADDRESS'])):
        e911_row = df_e911[(df_e911['E911 ADDRESS'] == address_dict[recorded_address]) & (recorded_zip == df_e911['GRAND LIST ZIPCODE'])]
        if len(e911_row) > 1:
            # print(row['Zip Code'], e911_row['GRAND LIST ZIPCODE'], e911_row)
            # We will update with the first as it looks like they are all duplicate names
            e911_row = e911_row.iloc[0]
            # raise Exception('More than 1 match')

        if len(e911_row) == 1:
            # e911_row = e911_row.squeeze()
            # print(recorded_address, recorded_zip)
            # print(e911_row)
            row['OWNER 1'] = e911_row['OWNER 1'].values[0]
            row['OWNER 2'] = e911_row['OWNER 2'].values[0]
            row['GRAND LIST ADDRESS '] = e911_row['GRAND LIST ADDRESS '].values[0]
            row['CAT'] = e911_row['CAT'].values[0]
            row['E911 ADDRESS'] = e911_row['E911 ADDRESS'].values[0]
            row['GRAND LIST = ADDRESS'] = 'T'

    out = (row['OWNER 1'], row['OWNER 2'], row['GRAND LIST ADDRESS '], row['CAT'], row['E911 ADDRESS'], row['GRAND LIST = ADDRESS'])

    return out

In [10]:
recorded_address = '16 ROBERTS AVE'
recorded_zip = '05760'
e911_row = df_e911[(df_e911['E911 ADDRESS'] == address_dict[recorded_address])]
e911_row


,OBJECTID_1,E911 ADDRESS,OBJECTID,OWNER 1,OWNER 2,GRAND LIST ADDRESS,GRAND LIST CITY,GRAND LIST STATE,GRAND LIST ZIPCODE,SPAN,...,GLVAL_HS,GLVAL_NR,CRHOUSPCT,MUNGL1PCT,AOEGL_HS,AOEGL_NR,SHAPESTAre,SHAPESTLen,Shape_Length,Shape_Area
13655,14938,16 ROBEROUTES AVE,18871879,BELL CATHERINE B,,208 DAVIS ROAD,CASTLETON,VT,05735,459-145-10073,...,0,1490,0,1490,0,1490,4316.297482,291.327087,291.327087,4316.297482


In [11]:
e911_row['OWNER 1'].values[0]

'BELL CATHERINE B'

In [12]:
len(main_df[~(main_df['GRAND LIST = ADDRESS'] == 'Y')])

1395

In [13]:
temp_df = main_df.copy()

columns_to_replace = ['OWNER 1', 'OWNER 2', 'GRAND LIST ADDRESS ', 'CAT', 'E911 ADDRESS', 'GRAND LIST = ADDRESS']
temp_df[columns_to_replace] = temp_df.apply(update_row, axis=1, result_type="expand")

In [14]:
len(temp_df[temp_df['GRAND LIST = ADDRESS'] == 'T'])

357

In [15]:
temp_df[temp_df['GRAND LIST = ADDRESS'] == 'T']

,OBJECTID,REF,ESITEID,Build Phase,ADDRESS,E911 ADDRESS,OWNER 1,OWNER 2,GRAND LIST ADDRESS,GRAND LIST = ADDRESS,...,COL_R,COL_S,COL_T,COL_U,COL_V,COL_W,COL_X,COL_Y,GNS Area,Construction Status
1105,707,707,78216,Phase 1,123 LAMBERT LN,123 LAMBEROUTE Ln,ROWE DANIEL J.,,123 LAMBERT LN,T,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,Excluded
1107,1082,1082,184342,Phase 1,28 ROBERTS AVE,28 ROBEROUTES AVE,WEYER DENNIS P,RIEDEL JANE E,28 ROBERTS AVE.,T,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,Excluded
1108,1075,1075,184338,Phase 1,37 ROBERTS AVE,37 ROBEROUTES AVE,FRAZIER DELBERT & BONITA,,37 ROBERTS AVE,T,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,Excluded
1109,1129,1129,184337,Phase 1,46 ROBERTS AVE,46 ROBEROUTES AVE,BURKE SYLVIA ANNE (LIFE ESTATE),,46 ROBERTS AVE,T,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,Excluded
1111,1173,1173,184692,Phase 1,56 ROBERTS AVE,56 ROBEROUTES AVE,DUNDON CHRISTOPHER,,RT 73,T,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,Excluded
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,695,695,316872,Phase 1,1691 VT ROUTE 125,1691 ROUTE 125,FELKL ROBERT & JOELLEN,,1691 RT 125,T,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,Excluded
2009,1073,1073,77968,Phase 1,1629 VT ROUTE 125,1629 ROUTE 125,JOBE BRION & FRANKS JUDITH,,105 COURT STREET,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Included
2012,320,320,77962,Phase 1,1390 VT ROUTE 125,1390 ROUTE 125,OGAWA NOBUO & CAVANAUGH CAROLE,,1390 RT 125,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Included
2016,1462,1462,77955,Phase 1,1011 VT ROUTE 125,1011 ROUTE 125,ELLERY KEITH & MELISSA,,1011 RT 125,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Included


In [16]:
temp_df.to_csv(DATA_DIR / 'processed' / 'temp_phase_1.csv', index=False)